In [2]:
import pandas as pd
import os
%xmode minimal

Exception reporting mode: Minimal


In [3]:
url = 'https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv'
file = '../data/titanic_2_original.pickle' # Damit es local gespeichert wird

if os.path.exists(file):
    t = pd.read_pickle(file)
else:
    t = pd.read_csv(url)
    t.to_pickle(file)
t.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
239,240,0,2,"Hunt, Mr. George Henry",male,33.0,0,0,SCO/W 1585,12.275,NaN,S
729,730,0,3,"Ilmakangas, Miss. Pieta Sofia",female,25.0,1,0,STON/O2. 3101271,7.925,NaN,S
593,594,0,3,"Bourke, Miss. Mary",female,NaN,0,2,364848,7.750,NaN,Q
134,135,0,2,"Sobey, Mr. Samuel James Hayden",male,25.0,0,0,C.A. 29178,13.000,NaN,S
819,820,0,3,"Skoog, Master. Karl Thorsten",male,10.0,3,2,347088,27.900,NaN,S
162,163,0,3,"Bengtsson, Mr. John Viktor",male,26.0,0,0,347068,7.775,NaN,S
585,586,1,1,"Taussig, Miss. Ruth",female,18.0,0,2,110413,79.650,E68,S
230,231,1,1,"Harris, Mrs. Henry Birkhardt (Irene Wallach)",female,35.0,1,0,36973,83.475,C83,S
497,498,0,3,"Shellard, Mr. Frederick William",male,NaN,0,0,C.A. 6212,15.100,NaN,S
612,613,1,3,"Murphy, Miss. Margaret Jane",female,NaN,1,0,367230,15.500,NaN,Q


## 4.1

In [4]:
def alter_bins(alter):
    if pd.isna(alter): return 'kein Alter'
    if alter < 11: return 'child'
    if alter <18 and alter >= 12: return 'teenager'
    if alter < 66 and alter > 17: return 'adult'
    if alter > 65: return 'adult'
    

In [5]:
t['alter_bin'] = t['Age'].apply(alter_bins)

In [6]:
t.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin
678,679,0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.00,1,6,CA 2144,46.9000,NaN,S,adult
258,259,1,1,"Ward, Miss. Anna",female,35.00,0,0,PC 17755,512.3292,NaN,C,adult
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.5000,NaN,S,child
288,289,1,2,"Hosono, Mr. Masabumi",male,42.00,0,0,237798,13.0000,NaN,S,adult
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.00,0,0,113572,80.0000,B28,NaN,adult


## 4.2

In [7]:
def alter_bins(alter,sex):
    if pd.isna(alter): return 'kein Alter'
    if alter < 11: return 'child'
    if alter <18 and alter >= 12: return 'teenager'
    if alter < 66 and alter > 17 and sex == 'female': return 'male'
    if alter < 66 and alter > 17 and sex == 'female': return 'male'
    if alter > 65: return 'adult'

In [8]:
def alter_bins(row):
    alter = row['Age']
    sex = row['Sex']

    if pd.isna(alter):
        return 'kein Alter'
    if alter < 11:
        return 'child'
    if 12 <= alter < 18:
        return 'teenager'
    if 18 <= alter < 66:
        return 'male' if sex == 'male' else 'female'  
    if alter >= 66:
        return 'adult'

# Anwenden der Funktion auf jede Zeile
t['alter_bin_sex'] = t.apply(alter_bins, axis=1)

In [9]:
t.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin,alter_bin_sex
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.00,B42,S,adult,female
822,823,0,1,"Reuchlin, Jonkheer. John George",male,38.00,0,0,19972,0.00,NaN,S,adult,male
58,59,1,2,"West, Miss. Constance Mirium",female,5.00,1,2,C.A. 34651,27.75,NaN,S,child,child
540,541,1,1,"Crosby, Miss. Harriet R",female,36.00,0,2,WE/P 5735,71.00,B22,S,adult,female
305,306,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.55,C22 C26,S,child,child


## 4.3
<br>
Funktionen: <br>
In Quantile aufteilen : t['Quantil'], grenzen = pd.qcut(Spalte bsp t['Spalte'], q=Anzahl Quantile, labels=Liste mit labels, retbins=True)

In [10]:
quantile_labels = ['niedrig', 'normal', 'hoch', 'sehr hoch']
t['Quantil'], grenzen = pd.qcut(t['Fare'], q=4, labels=quantile_labels, retbins=True)

# Ergebnis anzeigen
print(grenzen)

[  0.       7.9104  14.4542  31.     512.3292]


c)

In [11]:
median = grenzen[2]
median

14.4542

In [12]:
labels = ['billig', 'normal', 'teuer', 'sehr_teuer']

t_grouped_class = t.groupby('Pclass')

In [29]:
t_grouped_class['Fare'].sum()

Pclass
1    18177.4125
2     3801.8417
3     6714.6951
Name: Fare, dtype: float64

In [ ]:
t_grouped_class.iloc[0]

AttributeError: 'DataFrameGroupBy' object has no attribute 'iloc'

In [ ]:
# Definiere eine Funktion, die die Quantilschnitt (qcut) auf jede Gruppe anwendet
def quantile_cut(group):
    return pd.qcut(group, q=4, labels=labels, duplicates='drop')

# Gruppiere nach 'Pclass' und wende die Funktion auf die 'Fare'-Spalte an
t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(quantile_cut)

/var/folders/vr/klm94mws7lsd0vj7h5gd2bqw0000gn/T/ipykernel_57258/1720030728.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(quantile_cut)


In [ ]:
# Gruppiere nach 'Pclass' und wende die Funktion auf die 'Fare'-Spalte an
t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(lambda s: pd.qcut(s, q=4, labels=labels))

/var/folders/vr/klm94mws7lsd0vj7h5gd2bqw0000gn/T/ipykernel_57258/178223049.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(lambda s: pd.qcut(s, q=4, labels=labels))


## 4.4

In [16]:
def alone(row):
    if row == 0: return 0
    if row > 0: return 1
    if row == pd.NA: return 0

In [17]:
t['Alone'] = t['Parch'].apply(alone)

In [27]:
t.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin,alter_bin_sex,Quantil,Alone
672,673,0,2,"Mitchell, Mr. Henry Michael",male,70.0,0,0,C.A. 24580,10.5000,NaN,S,adult,adult,normal,0
231,232,0,3,"Larsson, Mr. Bengt Edvin",male,29.0,0,0,347067,7.7750,NaN,S,adult,male,niedrig,0
407,408,1,2,"Richards, Master. William Rowe",male,3.0,1,1,29106,18.7500,NaN,S,child,child,hoch,1
56,57,1,2,"Rugg, Miss. Emily",female,21.0,0,0,C.A. 31026,10.5000,NaN,S,adult,female,normal,0
495,496,0,3,"Yousseff, Mr. Gerious",male,NaN,0,0,2627,14.4583,NaN,C,kein Alter,kein Alter,hoch,0


In [31]:
t['QuantileFare'], grenzen= pd.qcut(t['Fare'], q=4, retbins=True)

In [32]:
t.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin,alter_bin_sex,Quantil,Alone,QuantileFare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,adult,male,niedrig,0,"(-0.001, 7.91]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adult,female,sehr hoch,0,"(31.0, 512.329]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,adult,female,normal,0,"(7.91, 14.454]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adult,female,sehr hoch,0,"(31.0, 512.329]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adult,male,normal,0,"(7.91, 14.454]"


In [ ]:
def alone(c):
    if c['Parch'] > 0 or c['SibSp'] > 0: return 0
    if c['Parch'] == 0 or c['SibSp'] == 0: return 1

0      0
1      0
2      0
3      0
4      0
      ..
886    0
887    0
888    2
889    0
890    0
Name: Parch, Length: 891, dtype: int64

In [ ]:
t['Alone'] = t.apply(alone, axis=1)